# Clustering with $k$-means and Gaussian mixtures

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [ ]:
np.random.seed(78787878)

## Data generation

In [ ]:
num_clusters = 3
num_samples = 200

X_unscaled, y = make_blobs(
    n_samples=num_samples,
    n_features=2,
    centers=num_clusters
)

print('Feature shape:', X_unscaled.shape)
print('Target shape:', y.shape)

In [ ]:
standard_scaler = StandardScaler()
X = standard_scaler.fit_transform(X_unscaled)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:,0], X[:,1], c=y, alpha=0.5, edgecolors='none')
ax.set_title('Generated data')
ax.set_aspect('equal', adjustable='datalim')
ax.set(xlabel='$x_1$', ylabel='$x_2$')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)

## $k$-means clustering

In [ ]:
max_components = 10

avg_silhouette_scores = []
for num_components in range(2, max_components+1):
    km_model = KMeans(n_clusters=num_components, n_init=10)
    km_model.fit(X)
    y_pred = km_model.predict(X)
    avg_silhouette_scores.append(silhouette_score(X, y_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.arange(2, max_components+1), avg_silhouette_scores, marker='o', clip_on=False, zorder=3)
ax.set_title('Model selection via silhouette scores')
ax.set(xlabel='no. of components', ylabel='average silhouette score', xlim=(0, max_components))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)

In [ ]:
num_components = np.argmax(avg_silhouette_scores) + 2
print('The model with {} clusters has been selected.'.format(num_components))

In [ ]:
km_model = KMeans(n_clusters=num_components, n_init=10)
y_pred = km_model.fit_predict(X)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:,0], X[:,1], c=y_pred, cmap='viridis', alpha=0.5, edgecolors='none', zorder=2)
ax.scatter(km_model.cluster_centers_[:,0], km_model.cluster_centers_[:,1], s=100, c='k', marker='x', zorder=3)
ax.set_aspect('equal', adjustable='datalim')
ax.apply_aspect()
X_values, Y_values = np.meshgrid(np.linspace(*ax.get_xlim(), 1001), np.linspace(*ax.get_ylim(), 1001))
Z_values = km_model.predict(np.stack((X_values.ravel(), Y_values.ravel()), axis=1)).reshape(X_values.shape)
ax.contourf(X_values, Y_values, Z_values, cmap='viridis', alpha=0.1, zorder=1)
ax.set_title('$k$-means clustering model')
ax.set(xlabel='$x_1$', ylabel='$x_2$')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)

## Gaussian mixture fitting

In [ ]:
max_components = 10

aics = []
bics = []
for num_components in range(1, max_components+1):
    gm_model = GaussianMixture(n_components=num_components)
    gm_model.fit(X)
    aics.append(gm_model.aic(X))
    bics.append(gm_model.bic(X))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.arange(1, max_components+1), aics, marker='o', label='AIC', clip_on=False, zorder=3)
ax.plot(np.arange(1, max_components+1), bics, marker='o', label='BIC', clip_on=False, zorder=3)
ax.set_title('Model selection via information criteria')
ax.set(xlabel='no. of components', ylabel='value', xlim=(0, max_components))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)

In [ ]:
num_components = np.argmin(bics) + 1
print('The model with {} components has been selected.'.format(num_components))

In [ ]:
gm_model = GaussianMixture(n_components=num_components)
y_pred = gm_model.fit_predict(X)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:,0], X[:,1], c=y_pred, cmap='viridis', alpha=0.5, edgecolors='none', zorder=2)
ax.set_aspect('equal', adjustable='datalim')
ax.apply_aspect()
X_values, Y_values = np.meshgrid(np.linspace(*ax.get_xlim(), 1001), np.linspace(*ax.get_ylim(), 1001))
Z_values = gm_model.score_samples(np.stack((X_values.ravel(), Y_values.ravel()), axis=1)).reshape(X_values.shape)
ax.contourf(X_values, Y_values, np.exp(Z_values), levels=10, cmap='Greys', alpha=0.7, zorder=1)
ax.set_title('Gaussian mixture model')
ax.set(xlabel='$x_1$', ylabel='$x_2$', )
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)